In [3]:
# ============================================================
# CloudAudit — Automated Configuration & Compliance Auditor
# ============================================================

import os
import json
import time
import random
from datetime import datetime

# Configuration
CONFIG_FILE = "data/configurations.json"
AUDIT_REPORT = "data/audit_report.json"

RULES = {
    "enable_encryption": True,
    "minimum_tls_version": "1.2",
    "public_access_disabled": True
}

# Ensure directories
os.makedirs(os.path.dirname(CONFIG_FILE), exist_ok=True)
os.makedirs(os.path.dirname(AUDIT_REPORT), exist_ok=True)

# Generate dummy configurations if not exist
if not os.path.exists(CONFIG_FILE):
    resources = []
    for i in range(1, 31):
        resources.append({
            "resource_id": f"res-{i:03}",
            "type": random.choice(["storage_bucket","compute_instance","database"]),
            "encryption_enabled": random.choice([True, False]),
            "tls_version": random.choice(["1.0","1.1","1.2","1.3"]),
            "public_access": random.choice([True, False])
        })
    with open(CONFIG_FILE, "w", encoding="utf-8") as f:
        json.dump(resources, f, indent=2)

def load_configs(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def audit_resource(res):
    findings = []
    if RULES["enable_encryption"] and not res.get("encryption_enabled", False):
        findings.append("encryption_disabled")
    if res.get("tls_version", "") < RULES["minimum_tls_version"]:
        findings.append("tls_version_too_low")
    if RULES["public_access_disabled"] and res.get("public_access", False):
        findings.append("public_access_allowed")
    return findings

def run_audit(path):
    configs = load_configs(path)
    report = {
        "timestamp": datetime.utcnow().isoformat(),
        "total_resources": len(configs),
        "compliant": 0,
        "non_compliant": 0,
        "failures": []
    }
    for res in configs:
        findings = audit_resource(res)
        if findings:
            report["non_compliant"] += 1
            report["failures"].append({
                "resource_id": res["resource_id"],
                "issues": findings
            })
        else:
            report["compliant"] += 1
    with open(AUDIT_REPORT, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)
    return report

if __name__ == "__main__":
    r = run_audit(CONFIG_FILE)
    print("Audit Report:", r)


Audit Report: {'timestamp': '2025-10-19T10:00:01.005493', 'total_resources': 30, 'compliant': 4, 'non_compliant': 26, 'failures': [{'resource_id': 'res-001', 'issues': ['encryption_disabled', 'public_access_allowed']}, {'resource_id': 'res-002', 'issues': ['encryption_disabled', 'tls_version_too_low']}, {'resource_id': 'res-003', 'issues': ['encryption_disabled', 'tls_version_too_low']}, {'resource_id': 'res-004', 'issues': ['encryption_disabled', 'tls_version_too_low']}, {'resource_id': 'res-005', 'issues': ['tls_version_too_low', 'public_access_allowed']}, {'resource_id': 'res-006', 'issues': ['encryption_disabled', 'public_access_allowed']}, {'resource_id': 'res-007', 'issues': ['encryption_disabled']}, {'resource_id': 'res-010', 'issues': ['tls_version_too_low']}, {'resource_id': 'res-011', 'issues': ['tls_version_too_low', 'public_access_allowed']}, {'resource_id': 'res-013', 'issues': ['encryption_disabled', 'tls_version_too_low', 'public_access_allowed']}, {'resource_id': 'res-0

/tmp/ipython-input-309092641.py:56: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
